# WWT Unravel 2025 — Final Hybrid Recommender Notebook

This notebook loads your CSVs, builds a hybrid recommender (co-occurrence + FP-Growth rules + Item2Vec embeddings + popularity), performs a light grid search to tune weights, and writes the submission Excel.

**IMPORTANT:** Upload `order_data.csv`, `customer_data.csv`, `store_data.csv`, and `test_data_question.csv` into `/content/sample_data` in Colab before running. If `order_data.csv` is large, runtime may take several minutes.

In [ ]:
# Install required packages (run once)
!pip install --quiet pandas numpy scikit-learn scipy openpyxl xlsxwriter tqdm mlxtend gensim annoy
print('Packages installed or already present.')

In [ ]:
# Imports and robust CSV reader
import os, json, re, random, shutil, math
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import sparse
from sklearn.preprocessing import MultiLabelBinarizer
print('Imports ok.')

def robust_read_csv(path):
    try:
        df = pd.read_csv(path, low_memory=False, engine='python', encoding='utf-8', on_bad_lines='skip')
        print(f'Loaded {path} ->', df.shape)
        return df
    except Exception as e:
        print('utf-8 failed:', e)
    try:
        df = pd.read_csv(path, low_memory=False, engine='python', encoding='latin1', on_bad_lines='skip')
        print(f'Loaded {path} with latin1 ->', df.shape)
        return df
    except Exception as e:
        print('latin1 failed:', e)
        raise

In [ ]:
# File paths (adjust if you stored files elsewhere)
order_path = '/content/sample_data/order_data.csv'
customer_path = '/content/sample_data/customer_data.csv'
store_path = '/content/sample_data/store_data.csv'
test_path = '/content/sample_data/test_data_question.csv'

print('Files exist and sizes:')
for p in [order_path, customer_path, store_path, test_path]:
    if os.path.exists(p):
        print(p, '->', f"{os.path.getsize(p)/(1024*1024):.2f} MB")
    else:
        print(p, '-> MISSING')

In [ ]:
# Load datasets
test_df = robust_read_csv(test_path)

order_df = None
customer_df = None
store_df = None

if os.path.exists(order_path):
    try:
        order_df = robust_read_csv(order_path)
    except Exception as e:
        print('Failed to load order_data.csv:', e)
else:
    print('order_data.csv not found; proceeding with test file only (lower accuracy expected).')

if os.path.exists(customer_path):
    try:
        customer_df = robust_read_csv(customer_path)
    except Exception as e:
        print('Failed to load customer_data.csv:', e)

if os.path.exists(store_path):
    try:
        store_df = robust_read_csv(store_path)
    except Exception as e:
        print('Failed to load store_data.csv:', e)

print('\nSummary shapes:')
print('order_df', None if order_df is None else order_df.shape)
print('customer_df', None if customer_df is None else customer_df.shape)
print('store_df', None if store_df is None else store_df.shape)
print('test_df', test_df.shape)

In [ ]:
# Parse and normalize transactions from order_df and test_df
def split_items(s):
    if pd.isna(s):
        return []
    s = str(s).strip()
    if s.startswith('[') and s.endswith(']'):
        try:
            arr = json.loads(s.replace("'", '"'))
            return [str(x).strip() for x in arr if str(x).strip()]
        except:
            pass
    for delim in ['|', '||', ';', ',', '\n', '/', ' + ', ' / ']:
        if delim in s:
            parts = [p.strip() for p in s.split(delim) if p.strip()]
            if len(parts) > 0:
                return parts
    return [s] if s else []

import re
def normalize_item(it):
    s = str(it).strip().lower()
    # remove common size tokens (customize to your needs)
    s = re.sub(r'\b(large|medium|small|xl|lrg|sm|20 oz|16 oz|12 oz|oz|ml|g|pack|pc|pcs)\b', '', s, flags=re.I)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

transactions = []

# From order_df if present
if order_df is not None and 'ORDERS' in order_df.columns:
    for s in order_df['ORDERS'].astype(str).values:
        parts = split_items(s)
        if parts:
            tx = [normalize_item(x) for x in parts if str(x).strip()]
            if tx:
                transactions.append(tx)
print('Transactions from order_df (if loaded):', len(transactions))

# From test_df item columns
item_cols = [c for c in test_df.columns if c.lower().startswith('item')]
print('Detected item columns in test file:', item_cols)
for _, row in test_df.iterrows():
    tx = []
    for c in item_cols:
        v = row.get(c)
        if pd.isna(v): continue
        s = str(v).strip()
        if s == '' or s.lower() == 'missing': continue
        tx.append(normalize_item(s))
    if tx:
        transactions.append(tx)

print('Total transactions prepared (order + test):', len(transactions))
print('Example transaction:', transactions[0] if transactions else 'NONE')

In [ ]:
# Build vocab and sparse transaction matrix in sparse format (memory efficient)
from collections import Counter
from scipy import sparse as sp
min_support = 1  # keep rare items to avoid losing test items; set to 2+ only if memory is critical
item_counts = Counter([it for tx in transactions for it in tx])
vocab = sorted([it for it, c in item_counts.items() if c >= min_support])
print('Vocab size:', len(vocab))

item_to_idx = {it: i for i, it in enumerate(vocab)}
idx_to_item = {i: it for it, i in item_to_idx.items()}

rows, cols = [], []
for r, tx in enumerate(transactions):
    for it in tx:
        if it in item_to_idx:
            rows.append(r)
            cols.append(item_to_idx[it])
data = np.ones(len(rows), dtype=np.uint8)
X_sparse = sp.csr_matrix((data, (rows, cols)), shape=(len(transactions), len(vocab)), dtype=np.uint8)
print('Transaction matrix shape:', X_sparse.shape, 'NNZ:', X_sparse.nnz)


In [ ]:
# Build top-k co-occurrence similarity lists (cosine-like)
import numpy as np
item_counts_arr = np.array(X_sparse.sum(axis=0)).ravel().astype(float)
n_items = X_sparse.shape[1]
sim_topk = 50
topk_sim_lists = {}
eps = 1e-9

for i in range(n_items):
    v = X_sparse[:, i]
    cooc = (X_sparse.T).dot(v).toarray().ravel()
    cooc[i] = 0.0
    denom = np.sqrt(item_counts_arr[i] * item_counts_arr + eps)
    sim = cooc / (denom + eps)
    top_idx = np.argsort(-sim)[:sim_topk]
    topk_sim_lists[i] = [(int(j), float(sim[j])) for j in top_idx if sim[j] > 0]

print('Built top-k similarity lists for items:', n_items)

In [ ]:
# Global popularity vector
item_counts_arr = np.array(X_sparse.sum(axis=0)).ravel().astype(float)
global_pop = item_counts_arr.copy()
if global_pop.max() > 0:
    global_pop = global_pop / global_pop.max()

# Store-level popularity (optional) - fallback to global_pop if store not present
store_pop = {}
if order_df is not None and 'STORE_NUMBER' in order_df.columns:
    groups = order_df.groupby('STORE_NUMBER')
    for store, g in groups:
        all_items = []
        col_candidates = [c for c in g.columns if 'order' in c.lower() or 'orders' in c.lower()]
        if col_candidates:
            col = col_candidates[0]
            for s in g[col].astype(str).values:
                parts = split_items(s)
                all_items.extend([normalize_item(x) for x in parts if str(x).strip()])
        cnt = Counter([it for it in all_items if it in item_to_idx])
        vec = np.array([cnt.get(idx_to_item[i], 0) for i in range(n_items)], dtype=float)
        if vec.max() > 0:
            store_pop[store] = vec / vec.max()
print('Store popularity vectors built for', len(store_pop), 'stores.')

In [ ]:
# Mine association rules using FP-Growth (mlxtend)
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_te = pd.DataFrame(te_ary, columns=te.columns_)

min_support_frac = 0.001  # tune as needed; lower -> more rules (and more compute)
freq = fpgrowth(df_te, min_support=min_support_frac, use_colnames=True)
rules = association_rules(freq, metric='confidence', min_threshold=0.25)
rules['antecedent'] = rules['antecedents'].apply(lambda x: frozenset([a.lower() for a in x]))
rules['consequent'] = rules['consequents'].apply(lambda x: [c.lower() for c in x])
rules_dict = {}
for _, r in rules.iterrows():
    rules_dict.setdefault(r['antecedent'], []).append((r['consequent'], float(r['confidence'])))
print('Extracted', len(freq), 'frequent itemsets and', len(rules), 'rules.')

In [ ]:
# Train Word2Vec (Item2Vec)
from gensim.models import Word2Vec
vec_size = 64
window = 5
min_count = 1
epochs = 12
workers = 2

sentences = transactions
w2v = Word2Vec(sentences=sentences, vector_size=vec_size, window=window, min_count=min_count, sg=1, negative=5, epochs=epochs, workers=workers)
print('Trained Word2Vec. Vocab size in model:', len(w2v.wv))

In [ ]:
# Build embedding matrix and normalize
from sklearn.preprocessing import normalize
n_items = len(vocab)
embed_dim = vec_size
item_emb_matrix = np.zeros((n_items, embed_dim), dtype=float)
for it, idx in item_to_idx.items():
    if it in w2v.wv:
        item_emb_matrix[idx] = w2v.wv[it]
    else:
        item_emb_matrix[idx] = np.random.normal(scale=0.01, size=(embed_dim,))
emb_norm = normalize(item_emb_matrix, axis=1)
print('Embedding matrix built and normalized.')

In [ ]:
# Embedding-based scores for a cart
def embedding_scores_for_cart(cart_items):
    cart_idxs = [item_to_idx[it] for it in cart_items if it in item_to_idx]
    if not cart_idxs:
        return np.zeros(n_items, dtype=float)
    cart_vecs = emb_norm[cart_idxs]
    sim_mat = emb_norm.dot(cart_vecs.T)
    avg_sim = sim_mat.mean(axis=1)
    if avg_sim.max() > 0:
        avg_sim = (avg_sim - avg_sim.min()) / (avg_sim.max() - avg_sim.min() + 1e-9)
    return avg_sim
print('Embedding score function ready.')

In [ ]:
# Hybrid recommender that combines similarity, rules, embedding, and popularity
def recommend_with_embeddings(cart_items, top_k=3, alpha=0.5, beta=0.2, gamma=0.2, store_number=None):
    cart_items = [it for it in cart_items if it in item_to_idx]
    cart_idxs = [item_to_idx[it] for it in cart_items]
    if len(cart_idxs) == 0:
        top_pop = np.argsort(-global_pop)[:top_k]
        return [idx_to_item[i] for i in top_pop]
    # similarity score
    score_sim = np.zeros(n_items, dtype=float)
    for ci in cart_idxs:
        for other_idx, s in topk_sim_lists.get(ci, []):
            score_sim[other_idx] += s
    if score_sim.max() > 0:
        score_sim = score_sim / (score_sim.max() + 1e-9)
    # rule score
    score_rule = np.zeros(n_items, dtype=float)
    cart_set = frozenset(cart_items)
    for ante, recs in rules_dict.items():
        if ante.issubset(cart_set):
            for cons, conf in recs:
                for c in cons:
                    if c in item_to_idx:
                        score_rule[item_to_idx[c]] = max(score_rule[item_to_idx[c]], conf)
    if score_rule.max() > 0:
        score_rule = score_rule / (score_rule.max() + 1e-9)
    # embedding score
    score_embed = embedding_scores_for_cart(cart_items)
    # popularity (store fallback)
    if store_number is not None and store_number in store_pop:
        pop_vec = store_pop[store_number]
    else:
        pop_vec = global_pop
    rem = max(0.0, 1.0 - (alpha + beta + gamma))
    combined = alpha * score_sim + beta * score_rule + gamma * score_embed + rem * pop_vec
    # exclude cart items
    for ci in cart_idxs:
        combined[ci] = -np.inf
    ranked = np.argsort(-combined)
    picks = []
    for idx in ranked:
        if combined[idx] == -np.inf:
            continue
        picks.append(idx_to_item[idx])
        if len(picks) >= top_k:
            break
    # fillers
    if len(picks) < top_k:
        fillers = [idx_to_item[i] for i in np.argsort(-global_pop) if idx_to_item[i] not in picks and idx_to_item[i] not in cart_items]
        for f in fillers:
            picks.append(f)
            if len(picks) >= top_k:
                break
    return picks

# Sanity test
print('Sanity rec for first tx:', transactions[0][:3], '->', recommend_with_embeddings(transactions[0][:3], 3))

In [ ]:
# Validation helper and grid search (light)
import itertools, random, time
def recall_at_k_model(sample_tx, k=3, alpha=0.5, beta=0.2, gamma=0.2, samples=1500):
    rng = random.Random(42)
    cand = [tx for tx in sample_tx if len(tx) >= 2]
    n = min(samples, len(cand))
    picks = rng.sample(cand, n)
    hits = 0
    for tx in picks:
        hidden = rng.choice(tx)
        observed = [t for t in tx if t != hidden]
        recs = recommend_with_embeddings(observed, k, alpha, beta, gamma)
        if hidden in recs:
            hits += 1
    return hits / n

alphas = [0.4, 0.5, 0.6]
betas  = [0.05, 0.1, 0.15]
gammas = [0.05, 0.1, 0.15]
best = (0, (0,0,0))
start = time.time()
for a,b,g in itertools.product(alphas, betas, gammas):
    if a + b + g >= 1.0: continue
    score = recall_at_k_model(transactions, 3, a, b, g, samples=1500)
    print(f'a={a}, b={b}, g={g} -> Recall@3: {score:.4f}')
    if score > best[0]:
        best = (score, (a,b,g))
print('\nBest validation Recall@3:', best, 'time elapsed:', time.time()-start)

In [ ]:
# Final predictions using best found params
best_score, best_params = best
best_a, best_b, best_g = best_params
print('Using best params:', best_a, best_b, best_g, 'val Recall@3:', best_score)

out_df = test_df.copy()
out_df['RECOMMENDATION 1'] = ''
out_df['RECOMMENDATION 2'] = ''
out_df['RECOMMENDATION 3'] = ''

for idx, row in tqdm(out_df.iterrows(), total=out_df.shape[0], desc='Generating recommendations'):
    cart_items = []
    for c in item_cols:
        v = row.get(c)
        if pd.isna(v): continue
        s = str(v).strip()
        if s == '' or s.lower() == 'missing': continue
        cart_items.append(normalize_item(s))
    recs = recommend_with_embeddings(cart_items, top_k=3, alpha=best_a, beta=best_b, gamma=best_g)
    for j in range(3):
        out_df.at[idx, f'RECOMMENDATION {j+1}'] = recs[j] if j < len(recs) else ''

out_name = 'Tom and Jerry_Recommendation_Output.xlsx'
out_df.to_excel(out_name, index=False)
print('Saved', out_name)